# Loop 10 Analysis: Precision Fix and Ensemble Strategy

## Key Issues to Address:
1. Experiment 009 achieved 70.659944 but failed on Kaggle due to overlapping trees in group 046
2. The evaluator identified precision loss as the root cause
3. Need to fix precision preservation and resubmit

In [1]:
import pandas as pd
import numpy as np
from decimal import Decimal, getcontext
getcontext().prec = 25

# Compare baseline vs ensemble submission
baseline = pd.read_csv('/home/code/experiments/001_baseline/santa-2025.csv', dtype=str)
ensemble = pd.read_csv('/home/code/experiments/009_full_ensemble_v2/submission.csv', dtype=str)

print('Baseline sample:')
print(baseline.head(10))
print('\nEnsemble sample:')
print(ensemble.head(10))

Baseline sample:
      id                       x                       y  \
0  001_0    s-48.196086194214246     s58.770984615214225   
1  002_0   s0.154097069621355887  s-0.038540742694794648   
2  002_1  s-0.154097069621372845  s-0.561459257305224058   
3  003_0      s1.123655816140301      s0.781101815992563   
4  003_1       s1.23405569584216      s1.275999500663759   
5  003_2      s0.641714640229075      s1.180458566613381   
6  004_0  s-0.324747789589372171   s0.132109978088185392   
7  004_1   s0.315354346242637695   s0.132109978063475492   
8  004_2   s0.324747789592379210  s-0.732109978069475531   
9  004_3  s-0.315354348134818330  s-0.732109978094185987   

                       deg  
0                    s45.0  
1  s203.629377730656841550  
2   s23.629377730656791812  
3        s111.125132292893  
4         s66.370622269343  
5      s155.13405193710082  
6  s156.370622145636389178  
7  s156.370622269264089255  
8  s336.370622269264003990  
9  s336.370622145636446021  

En

In [2]:
# Find differences between baseline and ensemble
diffs = []
for idx in baseline['id']:
    b_row = baseline[baseline['id'] == idx].iloc[0]
    e_row = ensemble[ensemble['id'] == idx].iloc[0]
    
    if b_row['x'] != e_row['x'] or b_row['y'] != e_row['y'] or b_row['deg'] != e_row['deg']:
        diffs.append({
            'id': idx,
            'baseline_x': b_row['x'],
            'ensemble_x': e_row['x'],
            'baseline_y': b_row['y'],
            'ensemble_y': e_row['y'],
            'baseline_deg': b_row['deg'],
            'ensemble_deg': e_row['deg']
        })

print(f'Found {len(diffs)} differences between baseline and ensemble')
if diffs:
    print('\nFirst 10 differences:')
    for d in diffs[:10]:
        print(f"  {d['id']}: x={d['baseline_x']} -> {d['ensemble_x']}")
        print(f"           y={d['baseline_y']} -> {d['ensemble_y']}")
        print(f"         deg={d['baseline_deg']} -> {d['ensemble_deg']}")
        print()

Found 20069 differences between baseline and ensemble

First 10 differences:
  004_0: x=s-0.324747789589372171 -> s-0.32474778959087557961
           y=s0.132109978088185392 -> s0.13210997809118560364
         deg=s156.370622145636389178 -> s156.37062214563638917753

  004_1: x=s0.315354346242637695 -> s0.31535434624113417579
           y=s0.132109978063475492 -> s0.13210997806647570285
         deg=s156.370622269264089255 -> s156.37062226926408925465

  004_2: x=s0.324747789592379210 -> s0.32474778959087557961
           y=s-0.732109978069475531 -> s-0.73210997806647526431
         deg=s336.370622269264003990 -> s336.37062226926400398952

  004_3: x=s-0.315354348134818330 -> s-0.31535434813632168272
           y=s-0.732109978094185987 -> s-0.73210997809118572022
         deg=s336.370622145636446021 -> s336.37062214563644602094

  006_0: x=s-0.504336377698653360 -> s-0.50602815502350417809
           y=s0.100031369878326948 -> s0.04589344637097881457
         deg=s293.62937773065658575

In [3]:
# Check group 046 specifically (the one that failed)
print('Group 046 in baseline:')
baseline_046 = baseline[baseline['id'].str.startswith('046_')]
print(baseline_046)

print('\nGroup 046 in ensemble:')
ensemble_046 = ensemble[ensemble['id'].str.startswith('046_')]
print(ensemble_046)

Group 046 in baseline:
          id                       x                       y  \
1035   046_0   s0.833509745263125934   s1.028869992073614847   
1036   046_1  s-0.145080334947463530   s0.174767026654041147   
1037   046_2   s0.085461679278546740  s-1.278514972467967414   
1038   046_3   s0.261478569890997048  s-1.554380905680907476   
1039   046_4   s0.225744842078161478  s-0.517947556258766872   
1040   046_5  s-1.061366436621962839  s-1.896511658504551612   
1041   046_6   s1.421973025969410775   s1.147989564559122044   
1042   046_7  s-1.136835696171881072  s-0.110887246914250615   
1043   046_8   s0.464592945641052524   s1.531094956749900149   
1044   046_9  s-1.362380148113562450  s-2.033679882973502107   
1045  046_10   s0.730963283991511559  s-0.649408671283886019   
1046  046_11  s-1.785694246697485665  s-1.281923573239253988   
1047  046_12   s1.669783885413652635   s1.526571425571550789   
1048  046_13  s-0.954742789708227790   s0.587250033712485298   
1049  046_14  s-1

In [4]:
# Check precision of values
def check_precision(val):
    val_str = str(val).replace('s', '')
    if '.' in val_str:
        return len(val_str.split('.')[1])
    return 0

print('Precision analysis for baseline:')
for col in ['x', 'y', 'deg']:
    precisions = baseline[col].apply(check_precision)
    print(f'  {col}: min={precisions.min()}, max={precisions.max()}, mean={precisions.mean():.1f}')

print('\nPrecision analysis for ensemble:')
for col in ['x', 'y', 'deg']:
    precisions = ensemble[col].apply(check_precision)
    print(f'  {col}: min={precisions.min()}, max={precisions.max()}, mean={precisions.mean():.1f}')

Precision analysis for baseline:
  x: min=14, max=55, mean=19.2
  y: min=15, max=54, mean=19.2
  deg: min=1, max=27, mean=19.0

Precision analysis for ensemble:
  x: min=14, max=20, mean=18.2
  y: min=15, max=20, mean=18.2
  deg: min=1, max=20, mean=18.2


In [5]:
# Check if there are any files with better scores that we haven't used
import glob
import os

# Find all CSV files
csv_files = glob.glob('/home/nonroot/snapshots/**/*.csv', recursive=True)
print(f'Found {len(csv_files)} CSV files in snapshots')

# Check which ones have valid format
valid_files = []
for f in csv_files:
    try:
        df = pd.read_csv(f, dtype=str, nrows=5)
        if 'id' in df.columns and 'x' in df.columns:
            valid_files.append(f)
    except:
        pass

print(f'Found {len(valid_files)} valid CSV files')

Found 760 CSV files in snapshots


Found 758 valid CSV files


In [6]:
# The key insight: we need to preserve EXACT string values when creating ensemble
# Let's create a proper ensemble that preserves precision

from shapely.geometry import Polygon
from shapely import affinity
from shapely.strtree import STRtree

# Tree geometry
TX = np.array([0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125])
TY = np.array([0, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5])
TY[0] = 0.8  # tip

def get_tree_polygon(x, y, deg):
    coords = list(zip(TX, TY))
    base_poly = Polygon(coords)
    rotated = affinity.rotate(base_poly, deg, origin=(0, 0))
    return affinity.translate(rotated, x, y)

def get_bounding_box_side(trees):
    if not trees:
        return float('inf')
    all_x, all_y = [], []
    for x, y, deg in trees:
        poly = get_tree_polygon(x, y, deg)
        bounds = poly.bounds
        all_x.extend([bounds[0], bounds[2]])
        all_y.extend([bounds[1], bounds[3]])
    return max(max(all_x) - min(all_x), max(all_y) - min(all_y))

def has_overlap(trees):
    if len(trees) <= 1:
        return False
    try:
        polygons = [get_tree_polygon(x, y, deg) for x, y, deg in trees]
        for i in range(len(polygons)):
            for j in range(i+1, len(polygons)):
                if polygons[i].intersects(polygons[j]) and not polygons[i].touches(polygons[j]):
                    return True
        return False
    except:
        return True

print('Functions defined')

Functions defined


In [7]:
# Check group 046 for overlaps
def parse_val(v):
    return float(str(v).replace('s', ''))

baseline_046 = baseline[baseline['id'].str.startswith('046_')]
trees_046 = []
for _, row in baseline_046.iterrows():
    x = parse_val(row['x'])
    y = parse_val(row['y'])
    deg = parse_val(row['deg'])
    trees_046.append((x, y, deg))

print(f'Group 046 has {len(trees_046)} trees')
print(f'Has overlap (baseline): {has_overlap(trees_046)}')
print(f'Bounding box side: {get_bounding_box_side(trees_046):.6f}')
print(f'Score: {get_bounding_box_side(trees_046)**2 / 46:.6f}')

# Check ensemble 046
ensemble_046 = ensemble[ensemble['id'].str.startswith('046_')]
trees_046_ens = []
for _, row in ensemble_046.iterrows():
    x = parse_val(row['x'])
    y = parse_val(row['y'])
    deg = parse_val(row['deg'])
    trees_046_ens.append((x, y, deg))

print(f'\nEnsemble 046 has {len(trees_046_ens)} trees')
print(f'Has overlap (ensemble): {has_overlap(trees_046_ens)}')

Group 046 has 46 trees
Has overlap (baseline): False
Bounding box side: 4.094486
Score: 0.364452

Ensemble 046 has 46 trees
Has overlap (ensemble): False


In [8]:
# The ensemble has different values for group 046 - let's see if they're from a different source
# and if the precision loss is causing the overlap

# Check the actual numeric differences
print("Comparing group 046 values:")
for i in range(46):
    b_row = baseline[baseline['id'] == f'046_{i}'].iloc[0]
    e_row = ensemble[ensemble['id'] == f'046_{i}'].iloc[0]
    
    b_x = parse_val(b_row['x'])
    e_x = parse_val(e_row['x'])
    b_y = parse_val(b_row['y'])
    e_y = parse_val(e_row['y'])
    b_deg = parse_val(b_row['deg'])
    e_deg = parse_val(e_row['deg'])
    
    if abs(b_x - e_x) > 1e-10 or abs(b_y - e_y) > 1e-10 or abs(b_deg - e_deg) > 1e-10:
        print(f"  Tree {i}: x diff={b_x - e_x:.15f}, y diff={b_y - e_y:.15f}, deg diff={b_deg - e_deg:.15f}")

Comparing group 046 values:
  Tree 0: x diff=-0.028088574377397, y diff=0.027866818240945, deg diff=-0.000000000047692
  Tree 1: x diff=-0.028088574380015, y diff=0.027866818244174, deg diff=-0.000000000845375
  Tree 2: x diff=-0.028088574377240, y diff=0.027866818240067, deg diff=0.000000000304865
  Tree 3: x diff=-0.028088574377276, y diff=0.027866818240170, deg diff=-0.000000000035243
  Tree 4: x diff=-0.028088574377131, y diff=0.027866818240171, deg diff=-0.000000000033594
  Tree 5: x diff=-0.028088574339179, y diff=0.027866818131218, deg diff=-0.000000032309686
  Tree 6: x diff=-0.028088574376554, y diff=0.027866818239615, deg diff=0.000000000328583
  Tree 7: x diff=-0.028088574427488, y diff=0.027866818229340, deg diff=-0.000000004540194
  Tree 8: x diff=-0.028088574377233, y diff=0.027866818240263, deg diff=0.000000000004405
  Tree 9: x diff=-0.028088574368917, y diff=0.027866818239988, deg diff=-0.000000000024144
  Tree 10: x diff=-0.028088574377820, y diff=0.027866818239371, d

In [9]:
# The key insight: the ensemble selected a DIFFERENT configuration for group 046
# that has overlaps according to Kaggle's stricter detection.
# We need to create a SAFE ensemble that only uses configurations that pass strict overlap checking.\n\n# Strategy:\n# 1. Use the baseline as the default (it's verified to work)\n# 2. For each N, check if any other source has a BETTER score AND passes strict overlap check\n# 3. Only replace if both conditions are met\n\nprint(\"Creating safe ensemble...\")\nprint(\"This will only replace baseline configs if the alternative is STRICTLY better and has no overlaps\")